In [1]:
import tensorflow as tf
import keras
import math
import numpy as np
import json

In [2]:
# construct the model
model = tf.keras.models.Sequential([

    # Convolutional Block 1
    tf.keras.layers.Conv2D(input_shape=(48, 48, 1),
        filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    # Convolutional Block 2
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    # convolutional Block 3
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=7, activation="softmax")    
  
])


### # display the model summary
model.summary()
### 
### # plot the model architecture
### tf.keras.utils.plot_model(
###     model, to_file='./images/architecture.png', show_shapes=True, show_dtype=False,
###     show_layer_names=True, rankdir='TB', expand_nested=False, dpi=150,
###     layer_range=None, show_layer_activations=False
### )

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                             

2023-02-14 18:47:27.293416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 18:47:27.293986: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
ds_path = r'/Users/heathsmith/repos/github/neuromorphic-computing/datasets/ck_plus_48.npz'

with np.load(ds_path) as data:
    images = data['images']
    labels = data['labels']
    classes = data['classes']
print(np.shape(images), np.shape(labels))

# constants
BATCH_SIZE = 16
_TRAIN = .80
_VAL = .10
_TEST = .10
SEED = 123

# create a dataset object and shuffle
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(len(images) * BATCH_SIZE, seed=SEED)
shuffle_images = []
shuffle_labels = []
for im, l in dataset.as_numpy_iterator():
    shuffle_images.append(im)
    shuffle_labels.append(l)
shuffle_images = np.asarray(shuffle_images)
shuffle_labels = np.asarray(shuffle_labels)
print(np.shape(shuffle_images))
print(len(dataset))

# create the slicing indices
TRAIN = round(len(dataset) * _TRAIN)
VAL = TRAIN + round(len(dataset) * _VAL)
TEST = VAL + round(len(dataset) * _TEST)
print(TRAIN, VAL, TEST)

x_train = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[0:TRAIN]))
x_train = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_train))
train_ds =  tf.data.Dataset.from_tensor_slices((x_train, shuffle_labels[0:TRAIN]))
train_ds = train_ds.cache()
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(x_train))

x_val = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[TRAIN:VAL]))
x_val = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_val))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, shuffle_labels[TRAIN:VAL]))
val_ds = val_ds.cache()
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(shuffle_images[TRAIN:VAL]))

x_test = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[VAL:TEST]))
x_test = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_test))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, shuffle_labels[VAL:TEST]))
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size=BATCH_SIZE,num_parallel_calls=tf.data.AUTOTUNE,drop_remainder=False)

(981, 48, 48) (981, 7)
(981, 48, 48)
981
785 883 981
(785, 48, 48, 1)
(98, 48, 48)


In [4]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.97
    epochs_drop = 10
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = tf.keras.optimizers.SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=sgd, metrics=['accuracy'])

In [5]:

# add checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    './simplified_cnn.h5',
    monitor='val_accuracy',
    verbose=0,
    save_best=True,
    save_weights_only=False,
    mode='max'
)

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[lr_sc, checkpoint], verbose=1)


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/25


2023-02-14 18:47:29.276330: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-14 18:47:29.280286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 2s 33ms/step - loss: 1.8727 - accuracy: 0.2051 - val_loss: 1.8428 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/25


2023-02-14 18:47:31.044113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 1s 20ms/step - loss: 1.8106 - accuracy: 0.2471 - val_loss: 1.8318 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/25
50/50 [==============================] - 1s 20ms/step - loss: 1.7921 - accuracy: 0.2764 - val_loss: 1.8150 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/25
50/50 [==============================] - 1s 21ms/step - loss: 1.7668 - accuracy: 0.3108 - val_loss: 1.7797 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/25
50/50 [==============================] - 1s 21ms/step - loss: 1.6835 - accuracy: 0.3771 - val_loss: 1.6115 - val_accuracy: 0.4592 - lr: 0.0100

Epoch 6: LearningRateScheduler setting learning rate to 0.01.
Epoch 6/25
50/50 [==============================] - 1s 20ms/step - loss: 1.3218 - accuracy: 0.5006 - val_loss: 1.2218 - val_accuracy: 0.5000 - 

In [6]:
#json.dump(history.history, open('./inception_history.json', 'w'))
print(history.history)

{'loss': [1.8727408647537231, 1.8105666637420654, 1.7920632362365723, 1.766776204109192, 1.6834803819656372, 1.3217594623565674, 0.9937416315078735, 0.7712110877037048, 0.6522316932678223, 0.420955091714859, 0.370577335357666, 0.30908462405204773, 0.1906367838382721, 0.2018159180879593, 0.19067253172397614, 0.13554224371910095, 0.09049773216247559, 0.07731524854898453, 0.06330408155918121, 0.07482918351888657, 0.14058762788772583, 0.09323330968618393, 0.126390278339386, 0.037945300340652466, 0.008889492601156235], 'accuracy': [0.2050955444574356, 0.2471337616443634, 0.2764331102371216, 0.3108280301094055, 0.3770700693130493, 0.5006369352340698, 0.6229299306869507, 0.7082802653312683, 0.7464967966079712, 0.8331210613250732, 0.8636943101882935, 0.8789808750152588, 0.9299362897872925, 0.9337579607963562, 0.9388535022735596, 0.9414013028144836, 0.9732484221458435, 0.9745222926139832, 0.9821656346321106, 0.9783439636230469, 0.9541401267051697, 0.9757962226867676, 0.9643312096595764, 0.98853

In [10]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# plot the results
epochs_range = np.arange(len(history.history['accuracy']))
fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))

fig.add_trace(
    go.Scatter(
        x=epochs_range, y=history.history['accuracy'],
        name='Training', legendgroup='accuracy', mode='lines',  line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_range, y=history.history['val_accuracy'], mode='lines',  line=dict(color='green'),
    name='Validation', legendgroup='accuracy'), row=1, col=1)

fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['loss'], line=dict(color='blue'),
    mode='lines', showlegend=False), row=1, col=2)
fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['val_loss'], line=dict(color='green'),
    mode='lines', showlegend=False), row=1, col=2)


fig.update_layout(
    template='plotly_dark',
    title="Simplified(1) - CK+ Dataset",
    title_x=0.5
    ## xaxis_title="Epochs"
    ## yaxis_title="LR Value",)
)

# save and display the plot image
pio.write_image(fig, "./history.png", scale=6, width=1000, height=600)
fig.show()

In [8]:
# evaluate the model
## loss, acc = model.evaluate(test_ds, verbose=2)

In [9]:
model.load_weights('./simplified_cnn.h5')
loss, acc = model.evaluate(test_ds, verbose=2)

7/7 - 0s - loss: 0.0732 - accuracy: 0.9694 - 70ms/epoch - 10ms/step
